In [1]:
#Barwieniw DAPI
import napari
from skimage import io
import os

# Path to the directory
data_directory = './'

# Initialize the viewer
viewer = napari.Viewer()

# Wczytanie obrazów
pliki = sorted([f for f in os.listdir(data_directory) if f.endswith('.tif')])
images = [io.imread(os.path.join(data_directory, plik)) for plik in pliki]

# Load images and add to viewer with different colors for each channel
colors = ['red', 'green', 'blue', 'magenta', 'cyan', 'yellow']
cnt = 0
for i, file_name in enumerate(sorted(os.listdir(data_directory))):
    if file_name.endswith('.tif'):
        image_path = os.path.join(data_directory, file_name)
        image = io.imread(image_path)
        name = "C" + str(cnt)
        viewer.add_image(image, name=name)
        viewer.layers[name].colormap = colors[cnt]
        cnt += 1

# Start the napari viewer
napari.run()

<tifffile.TiffFile 'C0.tif'> OME series failed to read '20190807_M01_001_Capture 1 - Capture 1 - B2 [144] Montage - 10_C1.tif', raised FileNotFoundError(2, 'No such file or directory'). Missing data are zeroed
<tifffile.TiffFile 'C0.tif'> OME series failed to read 250 files
<tifffile.TiffFile 'C1.tif'> OME series failed to read '20190807_M01_001_Capture 1 - Capture 1 - B2 [144] Montage - 10_C0.tif', raised FileNotFoundError(2, 'No such file or directory'). Missing data are zeroed
<tifffile.TiffFile 'C1.tif'> OME series failed to read 250 files
<tifffile.TiffFile 'C2.tif'> OME series failed to read '20190807_M01_001_Capture 1 - Capture 1 - B2 [144] Montage - 10_C0.tif', raised FileNotFoundError(2, 'No such file or directory'). Missing data are zeroed
<tifffile.TiffFile 'C2.tif'> OME series failed to read 250 files
<tifffile.TiffFile 'C3.tif'> OME series failed to read '20190807_M01_001_Capture 1 - Capture 1 - B2 [144] Montage - 10_C0.tif', raised FileNotFoundError(2, 'No such file or di

IndexError: list index out of range

In [ ]:
import napari
from tifffile import imread
import numpy as np


# Funkcja do załadowania obrazów TIFF i stworzenia listy danych obrazu
def load_images(channel_files):
    images = [imread(file) for file in channel_files]
    return images

# Nazwy plików z kanałami
channel_files = ['C0.tif', 'C1.tif', 'C2.tif', 'C3.tif', 'C4.tif', 'C5.tif']
nanes = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5']

# Załaduj obrazy
images = load_images(channel_files)

# Kolory dla kanałów
channel_colors = ['red', 'green', 'blue', 'magenta', 'yellow', 'cyan']

# Uruchom Napari
viewer = napari.Viewer()

# Dodaj obrazy do Napari
for image, color, name in zip(images, channel_colors, nanes):
    viewer.add_image(image, colormap=color, blending='additive', name=name)

# Pokaż wizualizację
napari.run()

In [ ]:
# Zwykła segmentacja za pomocą parametrów obrazu

import napari
from tifffile import imread
from scipy.ndimage import median_filter, gaussian_filter
import numpy as np
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square, remove_small_objects, opening
from skimage.segmentation import clear_border
from skimage.measure import label
from skimage import measure
from apoc import ObjectSegmenter


def segment(image):
    # threshold Otsu
    thresh = threshold_otsu(image)
    bw = closing(image > thresh, square(3)) 

    cleared = clear_border(bw)
    
    # usunięcie małych obiektów
    cleared = remove_small_objects(cleared, min_size=8000)

    # otwarcie morfologiczne
    cleared = opening(cleared, square(5))

    label_image = label(cleared)
    
    # pomiar obszarów
    properties = measure.regionprops(label_image)

    final_labels = np.zeros_like(label_image)
    
    for prop in properties:
        # Sprawdzenie, czy obiekt jest wystarczająco okrągły (ekscentryczność < 0.85)
        # oraz czy jest zwarty (solidity > 0.95)
        if prop.eccentricity < 0.85 and prop.solidity > 0.95:
            minr, minc, maxr, maxc = prop.bbox
            # Sprawdzenie, czy obiekt jest w pełni zawarty w granicach obrazu
            if minr > 0 and minc > 0 and maxr < label_image.shape[0] and maxc < label_image.shape[1]:
                # Przypisanie etykiety do finalnego obrazu
                final_labels[label_image == prop.label] = prop.label

    return final_labels

# Załaduj obraz do segmentacji
c1_file = 'C1.tif'
C1_img = imread(c1_file)

# Wybór warstwy
C1_img = C1_img[1]
C1_img = C1_img[10:35]

C1_img_gaussian_filtered = []

for img in C1_img:
    filtered_img = gaussian_filter(img, sigma=9)
    filtered_img = median_filter(filtered_img, size=12)
    C1_img_gaussian_filtered.append(filtered_img)

C1_img_gaussian_filtered = np.array(C1_img_gaussian_filtered)

segmented_cells = []
# Segmentuj komórki
for img in C1_img_gaussian_filtered:
    segmented_cells.append(segment(img))

segmented_cells= np.array(segmented_cells)
# Uruchom Napari
viewer = napari.Viewer()

# Dodaj obrazy do Napari
viewer.add_image(C1_img, colormap='green', blending='additive', name='C1 Layer Original')
viewer.add_labels(segmented_cells, name='Segmented Cells')

# Pokaż wizualizację
napari.run()

In [ ]:
# Segmentacja sieci neuronowe
import napari
from tifffile import imread
from scipy.ndimage import median_filter, gaussian_filter
import numpy as np
from skimage.filters import threshold_otsu
from skimage.morphology import closing, opening, square, remove_small_objects
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from apoc import ObjectSegmenter
from skimage import measure
import skimage.morphology

# Uruchom Napari
viewer = napari.Viewer()

def remove_joined_and_incomplete_cells(labels):
    # Usunięcie małych obiektów
    labels = remove_small_objects(labels, min_size=8000)
    # Otwarcie morfologiczne w celu poprawy segmentacji
    labels = opening(labels, square(3))
    # Pobranie właściwości obiektów
    properties = regionprops(labels)
    final_labels = np.zeros_like(labels)

    for prop in properties:
        # Sprawdzenie, czy obiekt jest wystarczająco okrągły (ekscentryczność < 0.85)
        # oraz czy jest zwarty (solidity > 0.95)
        if prop.eccentricity < 0.85 and prop.solidity > 0.95:
            minr, minc, maxr, maxc = prop.bbox
            # Sprawdzenie, czy obiekt jest w pełni zawarty w granicach obrazu
            if minr > 0 and minc > 0 and maxr < labels.shape[0] and maxc < labels.shape[1]:
                # Przypisanie etykiety do finalnego obrazu
                final_labels[labels == prop.label] = prop.label

    return final_labels

c1_file = 'C1.tif'
C1_img = imread(c1_file)

# Wybór warstwy
C1_img = C1_img[1]
C1_img = C1_img[10:35]

C1_img_gaussian_filtered = []

# Filtracja do nauki
for img in C1_img:
    filtered_img = gaussian_filter(img, sigma=9)
    filtered_img = median_filter(filtered_img, size=12)
    C1_img_gaussian_filtered.append(filtered_img)

C1_img_gaussian_filtered = np.array(C1_img_gaussian_filtered)
# Dodaj obrazy do Napari
viewer.add_image(C1_img_gaussian_filtered, colormap='green', blending='additive', name='C1 Layer all')

# Wczytanie obrazu z zaznaczoną komórką
manual_annotations = imread('layer2.tif')
# Dodaj obraz do Napari
viewer.add_image(manual_annotations, name='label')
# Utworzenie obiektu
segmenter = ObjectSegmenter(opencl_filename='./ObjectSegmenter.cl')

result = []
for img in C1_img:
    # Trening sieci
    segmenter.train("sobel_of_gaussian_blur=2 laplace_box_of_gaussian_blur=2 gaussian_blur=2 sobel_of_gaussian_blur=4 laplace_box_of_gaussian_blur=4 gaussian_blur=4", 
                manual_annotations, img)    
    # Predykcja
    result.append(segmenter.predict(image=img))

result = np.array(result)

# Dodaj obraz segmentacji do Napari
viewer.add_labels(result, name='Segmented Cells')
# Konwersja wyniku na warstwę
layer_result = np.array(result)

finaly = []

for layer in layer_result:
    finaly.append(remove_joined_and_incomplete_cells(layer))

finaly = np.array(finaly)

# Dodaj obraz segmentacji do Napari
viewer.add_labels(finaly, name='Final segmented Cells')

# Pokaż wizualizację
napari.run()


In [ ]:
# Odchylenie standardowe tła
import napari
from tifffile import imread
from scipy.ndimage import median_filter, gaussian_filter
import numpy as np

# Wczytanie obrazu
c1_file = 'C1.tif'
c1_image = imread(c1_file)

# Wybór warstwy
layer = c1_image[1]
layer = layer[23]

# Filtracja medianowa i gaussowska
layer_gaussian_filtered = gaussian_filter(layer, sigma=9)
layer_median_filtered = median_filter(layer, size = 12)

# Porównanie metod
std_img = np.std(layer[0:430, 480:1300])
std_nlm = np.std(layer_median_filtered[0:430, 480:1300])
std_gauss = np.std(layer_gaussian_filtered[0:430, 480:1300])

print('\n', end="")
print(f"Odchylenie standardowe tła oryginalnego obrazu      : {std_img}")
print(f"Odchylenie standardowe tła po filtracji medianowej  : {std_nlm}")
print(f"Odchylenie standardowe tła po filtracji gausowskiej : {std_gauss}")
print('\n', end="")

# Uruchom Napari
viewer = napari.Viewer()

# Dodaj obrazy do Napari
viewer.add_image(layer, colormap='green', blending='additive', name='C1 Layer Original')
viewer.add_image(layer_gaussian_filtered, colormap='cyan', blending='additive', name='C1 Layer Gaussian Filtered')
viewer.add_image(layer_median_filtered, colormap='yellow', blending='additive', name='C1 Layer Median Filtered')

In [ ]:
import matplotlib.pyplot as plt
from skimage.io import imread

# Wczytanie obrazu
file_path = 'med.tif'
image = imread(file_path)

# Wyświetlenie obrazu za pomocą Matplotlib
plt.figure(figsize=(8, 8))
plt.imshow(image, cmap='gray')
plt.axis('off')  # Wyłączenie osi
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Dane
data = [5.281590046, 4.294470816, 3.992205868, 3.226203246, 4.898898645, 4.590090315, 4.330786034, 4.860282294, 4.708028946, 4.093988319, 3.58067422]

# Parametry statystyczne
mean = np.mean(data)
std_dev = np.std(data, ddof=1)

print('Średnia objętość jąder komórkowych: ', mean)
print('Odchylenie standardowe:             ', std_dev)

# Histogram
plt.figure(figsize=(12, 6))

plt.hist(data, bins='auto', edgecolor='black')
plt.title('Histogram')
plt.xlabel('V [\u00B5m]')
plt.ylabel('Frequency')
plt.axvline(mean, color='r', linestyle='dashed', linewidth=1, label=f'Mean: {mean:.2f}')
plt.axvline(np.percentile(data, 25), color='g', linestyle='dashed', linewidth=1, label='25th percentile')
plt.axvline(np.percentile(data, 75), color='b', linestyle='dashed', linewidth=1, label='75th percentile')
plt.legend()
plt.show()

# Boxplot
plt.figure(figsize=(12, 6))
plt.boxplot(data, vert=False)
plt.title('Boxplot')
plt.xlabel('V [\u00B5m]')

plt.tight_layout()
plt.show()